# Spark Train Logistic Regression


Train Logistic Regression classifier with Apache SparkML

In [1]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

echo $version

if [ $version == '36' ] || [ $version == '37' ]; then
    echo 'Starting installation...'
    pip3 install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
elif [ $version == '38' ] || [ $version == '39' ]; then
    pip3 install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
else
    echo 'Currently only python 3.6, 3.7 , 3.8 and 3.9 are supported, in case you need a different version please open an issue at https://github.com/IBM/claimed/issues'
    exit -1
fi

37
Starting installation...
Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)


In [2]:
from pyspark import SparkContext, SparkConf, SQLContext
import os
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import logging
import shutil
import site
import sys
import wget
import re

In [3]:
if sys.version[0:3] == '3.9':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.8':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.7':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
elif sys.version[0:3] == '3.6':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
else:
    raise Exception('Currently only python 3.6 , 3.7, 3,8 and 3.9 is supported, in case '
                    'you need a different version please open an issue at '
                    'https://github.com/IBM/claimed/issues')

In [4]:
data_parquet = os.environ.get('data_parquet',
                              'data.parquet')  # input file name (parquet)
master = os.environ.get('master',
                        "local[*]")  # URL to Spark master
model_target = os.environ.get('model_target',
                              "model.xml")  # model output file name
data_dir = os.environ.get('data_dir',
                          '../../data/')  # temporary directory for data
input_columns = os.environ.get('input_columns',
                               '["x", "y", "z"]')  # input columns to consider

In [5]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [7]:
conf = SparkConf().setMaster(master)
#if sys.version[0:3] == '3.6' or sys.version[0:3] == '3.7':
conf.set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')

sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [8]:
df = spark.read.parquet(data_dir + data_parquet)

In [9]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [10]:
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

In [11]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [12]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [21]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr2 = LogisticRegression(maxIter=10, regParam=0.01, elasticNetParam=0.0)
lr3 = LogisticRegression(maxIter=100, regParam=0.5, elasticNetParam=0.5)
lr4 = LogisticRegression(maxIter=1000, regParam=2.0, elasticNetParam=1.0)

In [25]:
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])
pipeline2 = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr2])
pipeline3 = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr3])
pipeline4 = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr4])

In [26]:
model = pipeline.fit(df_train)
model2 = pipeline2.fit(df_train)
model3 = pipeline3.fit(df_train)
model4 = pipeline4.fit(df_train)

In [27]:
prediction = model.transform(df_train)
prediction2 = model2.transform(df_train)
prediction3 = model3.transform(df_train)
prediction4 = model4.transform(df_train)

In [37]:
binEval = MulticlassClassificationEvaluator(). \
    setMetricName("accuracy"). \
    setPredictionCol("prediction"). \
    setLabelCol("label")

print("maxIter=10, regParam=0.3, elasticNetParam=0.8 Accuracy:",binEval.evaluate(prediction))
print("maxIter=10, regParam=0.01, elasticNetParam=0.0 Accuracy:",binEval.evaluate(prediction2))
print("maxIter=100, regParam=0.5, elasticNetParam=0.5 Accuracy:",binEval.evaluate(prediction3))
print("maxIter=1000, regParam=2.0, elasticNetParam=1.0 Accuracy:",binEval.evaluate(prediction4))

maxIter=10, regParam=0.3, elasticNetParam=0.8 Accuracy: 0.20729283314669653


maxIter=10, regParam=0.01, elasticNetParam=0.0 Accuracy: 0.32902015677491603


maxIter=100, regParam=0.5, elasticNetParam=0.5 Accuracy: 0.20729283314669653


[Stage 373:===============================================>       (12 + 2) / 14]

maxIter=1000, regParam=2.0, elasticNetParam=1.0 Accuracy: 0.20729283314669653


In [40]:
splits2 = df.randomSplit([0.7, 0.3, seed=1])
splits3 = df.randomSplit([0.9, 0.1])
df_train1 = splits2[0]
df_test1 = splits2[1]
df_train2 = splits3[0]
df_test2 = splits3[1]

SyntaxError: invalid syntax (2493712700.py, line 1)

In [39]:
model5 = pipeline4.fit(df_train1)
model6 = pipeline4.fit(df_train2)
prediction5 = model5.transform(df_train1)
prediction6 = model6.transform(df_train2)
print(".7/.3 Test Split Accuracy:",binEval.evaluate(prediction5))
print(".9/.1 Test Split Accuracy:",binEval.evaluate(prediction6))

.7/.3 Test Split Accuracy: 0.20679368005940948


[Stage 461:===============================================>       (12 + 2) / 14]

.9/.1 Test Split Accuracy: 0.20684228583361924


In [ ]:
pmmlBuilder = PMMLBuilder(sc, df_train, model)
pmmlBuilder.buildFile(data_dir + model_target)

'/resources/labs/BD0231EN/claimed/component-library/train/../../data/model.xml'